# Run alternative flux modes simulations

In [1]:
# | default_exp runalternativemodes

As with runconstraintscan.py, this notebook can either be run as a notebook, or using nbdev a python script is produced that can be run in the command line for easier running on e.g. a cluster. It is a notebook in the nbs directory that is symlinked in the nbs/functions directory in order to be exported as a .py file in the mmon-gcm library.

NOTE: I have had issues where for an unknown reason the code hangs while solving. In this case I can check the temporary files to see which solutions are missing and run it again to just get those. For the paper, the blue light, unconstrained, KO solution number 348 hangs with a tolerance of 1e-8 and says infeasible for a tolerance of 1e-7. So the solution list is constructed from combining the remaining temporary files rather than all in one go. 

In [2]:
# | export

import os
import shutil
import sys
from pathlib import Path

import cobra

# from x import y syntax doesn't work because of nbdev export format
import mmon_gcm.alternativemodes
import mmon_gcm.buildingediting
import mmon_gcm.supermodel
import pandas as pd
from pandarallel import pandarallel

In [3]:
# This cell isn't exported to the .py file, so define here if running in notebook rather than as .py on e.g.a cluster

sys.argv = [
    "script_name",
    "../outputs/alternative_weighting/test_solution.csv",
    "../models/4_stage_GC.json",
    "../outputs/alternative_weighting/alternative_weights.csv",
    "../inputs/arabidopsis_parameters.csv",
    "blue",
    "False",
    "True",
    "2",
]

In [4]:
# | export

results_path = sys.argv[1]
model_path = sys.argv[2]
weightings_csv = sys.argv[3]
parameters_csv = sys.argv[4]
light_colour = sys.argv[5]
atpase_constraint = sys.argv[6]
starch_knockout = sys.argv[7]
no_cores = int(sys.argv[8])

In [5]:
# | export

if light_colour != "blue" and light_colour != "white" and light_colour != "nops":
    raise ValueError(
        f"Please specify either 'blue' or 'white' or 'nops' for light, not {light_colour}"
    )

if atpase_constraint == "True":
    atpase_constraint = True
elif atpase_constraint == "False":
    atpase_constraint = False
else:
    raise ValueError(
        f"Please specify True or False for the ATPase constraint, not {atpase_constraint}"
    )

if starch_knockout == "True":
    starch_knockout = True
elif starch_knockout == "False":
    starch_knockout = False
else:
    raise ValueError(
        f"Please specify True or False for the starck knockout, not {starch_knockout}"
    )

In [6]:
# | export

model = cobra.io.load_json_model(model_path)
print("Model imported")

Model imported


In [9]:
model.reactions.GLC_c_gc_Linker_2

Reaction identifier,GLC_c_gc_Linker_2
Name,GLC_c_gc_Linker_2
Memory address,0x7ff5e4fd9310
Stoichiometry,2.0 GLC_c_gc_2 --> 0.08695652173913043 GLC_c_gc_3 + GLC_total_pseudometabolite_2 + pseudoOs_c_gc_2 2.0 GLC_gc_2 --> 0.08695652173913043 GLC_gc_3 + GLC_total_pseudometabolite_2 + pseudoOs_c_gc_2
GPR,
Lower bound,0.0
Upper bound,1000.0


In [6]:
# | export

print(model.solver.configuration.tolerances.integrality)
print(model.solver.configuration.tolerances.feasibility)
model.solver.configuration.tolerances.feasibility = (
    1e-8  # 1e-9 takes a long time to solve
)
print(model.solver.configuration.tolerances.feasibility)

1e-07
1e-07
1e-08


In [7]:
# | export

parameters_df = pd.read_csv(parameters_csv, index_col=0)

In [8]:
parameters_df

,Value,Units,Source
P_abs,9.000000e-01,Dimensionless,"Zhu, Long, and Ort (2010)"
T_l,1.700000e-04,m,Wuyts et al. (2010)
A_l,1.000000e+00,m$^2$,Fixed
V_gc_ind,4.750000e-13,dm$^3$,Jezek and Blatt (2017)
FqFm,9.000000e-01,Dimensionless,Lawson (2003)
R_ch,6.923077e-02,Dimensionless,"Fujiwara, Sanjaya, and Itoh (2019)"
L_air,3.700000e-01,Dimensionless,Earles et al. (2018)
L_epidermis,1.500000e-01,Dimensionless,Wuyts et al. (2010)
Vac_frac,7.510000e-01,Dimensionless,Wang et al. (2017)
T,2.961500e+02,K,Horrer et al. (2016)


In [9]:
# | export

arabidopsis_supermodel = mmon_gcm.supermodel.SuperModel(
    parameters_df.loc[:, "Value"], fba_model=model
)
arabidopsis_supermodel.constrain_osmolarity(printouts=False)
arabidopsis_supermodel.constrain_photons(150, printouts=False)
arabidopsis_supermodel.add_maintenance();

## Constrain light, ATPase, starch

In [10]:
# | export

if light_colour == "blue":
    arabidopsis_supermodel.fba_model.reactions.Photon_tx_gc_2.upper_bound = 0
    arabidopsis_supermodel.fba_model.reactions.Photon_tx_me_2.upper_bound = 0
    print("Model constrained with blue light")
elif light_colour == "nops":
    mmon_gcm.buildingediting.set_bounds_multi(
        arabidopsis_supermodel.fba_model, "Photon_tx_gc", 0, 0
    )
    print("Photosynthesis prevented in guard cell")
else:
    print("Model constrained with white light")

if atpase_constraint == True:
    gc_atpase_upper_bound = arabidopsis_supermodel.get_atpase_constraint_value(7.48)
    mmon_gcm.buildingediting.set_bounds_multi(
        arabidopsis_supermodel.fba_model, "PROTON_ATPase_c_gc", 0, gc_atpase_upper_bound
    )
    print("Model ATPase constrained")
else:
    print("Model ATPase left unconstrained")

if starch_knockout == True:
    mmon_gcm.buildingediting.set_bounds_multi(
        arabidopsis_supermodel.fba_model, "RXN_1827_p_gc", 0, 0
    )
    print("Model starch knocked out")
else:
    print("Model starch left unconstrained")

print("Supermodel established and model constrained")

Model constrained with blue light
Model ATPase left unconstrained
Model starch knocked out
Supermodel established and model constrained


In [13]:
# | export
weightings = pd.read_csv(weightings_csv, index_col=[0], header=[0])
print("Weightings file imported")

Weightings file imported


**If you are running from notebook you'll want to comment out the next cell! It's here so that running nbdev tests doesn't take too long, as they test running the entire notebook. It won't be exported so running as a script will be fine**

In [12]:
weightings = weightings.iloc[0:4]  # so tests don't take too long
weightings.head()

,10_FORMYL_THF_pc_gc_1,10_FORMYL_THF_pc_gc_2,10_FORMYL_THF_pc_gc_3,10_FORMYL_THF_pc_gc_4,10_FORMYL_THF_pc_me_1,10_FORMYL_THF_pc_me_2,10_FORMYL_THF_pc_me_3,10_FORMYL_THF_pc_me_4,1TRANSKETO_RXN_p_gc_1,1TRANSKETO_RXN_p_gc_2,...,sSUC_biomass_me_3,sSUC_biomass_me_4,unlProtHYPO_c_gc_1,unlProtHYPO_c_gc_2,unlProtHYPO_c_gc_3,unlProtHYPO_c_gc_4,unlProtHYPO_c_me_1,unlProtHYPO_c_me_2,unlProtHYPO_c_me_3,unlProtHYPO_c_me_4
0,3.196322,0.266360,6.126284,3.196322,2.003810,0.166984,3.840636,2.003810,4.628290,0.385691,...,3.685919,1.923088,3.013075,0.251090,5.775060,3.013075,4.919640,0.409970,9.429310,4.919640
1,2.405191,0.200433,4.609950,2.405191,2.976393,0.248033,5.704753,2.976393,5.442639,0.453553,...,10.874785,5.673801,1.602658,0.133555,3.071762,1.602658,4.092069,0.341006,7.843133,4.092069
2,5.692358,0.474363,10.910353,5.692358,4.656864,0.388072,8.925656,4.656864,4.362721,0.363560,...,6.496164,3.389303,2.826699,0.235558,5.417839,2.826699,3.024736,0.252061,5.797410,3.024736
3,3.247298,0.270608,6.223988,3.247298,3.545171,0.295431,6.794911,3.545171,2.244889,0.187074,...,8.896234,4.641513,2.847609,0.237301,5.457916,2.847609,2.109259,0.175772,4.042746,2.109259


First let's check for a temporary directory to see if we already have some solutions:

In [25]:
# | export

temp_results = Path(results_path).parent / "_tmp/"

In [30]:
# | export


def get_file_names_as_integers(directory_path):
    try:
        # Get a list of all files and directories in the specified directory
        file_list = os.listdir(directory_path)

        # Filter out directories and keep only file names
        file_names_as_integers = [
            int(os.path.splitext(filename)[0])
            for filename in file_list
            if os.path.isfile(os.path.join(directory_path, filename))
        ]

        return file_names_as_integers
    except (OSError, ValueError) as e:
        print(f"An error occurred: {e}")
        return []

In [36]:
# | export

if os.path.isdir(temp_results):
    print(f"Already a tmp directory at {temp_results}, checking for existing solutions")
    existing_solutions = get_file_names_as_integers(temp_results)
    print(
        f"There are already {len(existing_solutions)} solutions, dropping these from the weightings csv"
    )

    weightings = weightings.drop(existing_solutions)

else:
    print(
        f"No existing tmp directory at {temp_results}, creating temp directory to store solutions as they come in"
    )
    Path(temp_results).mkdir(parents=True, exist_ok=True)

Already a tmp directory at ../outputs/alternative_weighting/_tmp, checking for existing solutions
There are already 645 solutions, dropping these from the weightings csv


KeyError: '[528, 758, 163, 216, 677, 45, 104, 620, 196, 77, 772, 557, 126, 299, 648, 264, 587, 809, 812, 849, 559, 751, 107, 204, 103, 280, 178, 591, 314, 646, 766, 269, 145, 55, 89, 791, 229, 90, 652, 530, 602, 511, 84, 13, 186, 796, 114, 10, 806, 215, 298, 764, 860, 670, 64, 624, 853, 172, 58, 39, 326, 327, 310, 816, 833, 625, 211, 283, 61, 507, 682, 43, 838, 595, 54, 236, 799, 221, 820, 613, 294, 253, 512, 308, 8, 855, 336, 60, 194, 768, 580, 271, 522, 19, 655, 815, 69, 544, 586, 824, 828, 31, 571, 590, 81, 788, 500, 140, 623, 795, 292, 329, 632, 11, 579, 14, 645, 173, 232, 598, 59, 158, 567, 56, 304, 866, 16, 761, 249, 626, 515, 199, 854, 857, 553, 513, 569, 588, 518, 258, 823, 514, 225, 519, 202, 548, 313, 309, 49, 606, 207, 868, 546, 802, 324, 152, 195, 200, 575, 53, 601, 814, 862, 750, 790, 577, 37, 32, 183, 817, 572, 536, 94, 539, 678, 65, 165, 785, 504, 119, 669, 251, 133, 209, 102, 770, 753, 765, 113, 604, 291, 858, 281, 661, 543, 657, 585, 568, 275, 782, 671, 859, 110, 130, 

In [ ]:
# | export

pandarallel.initialize(nb_workers=no_cores, progress_bar=False)
print(f"Solving model for {len(weightings.index)} alternative weightings")
weightings_solution = weightings.parallel_apply(
    mmon_gcm.alternativemodes.solve_model_with_weightings,
    args=([arabidopsis_supermodel.fba_model, temp_results]),
    axis=1,
)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Solving model for 4 alternative weightings
Solving solution 0
Solving solution 2
Solving solution 3
Solving solution 1


In [ ]:
# | export

weightings_solution.to_csv(results_path)
if len(weightings_solution) == len(weightings):
    print(f"All solutions saved to {results_path}")
    print(f"Deleting temp directory {temp_results}")
    shutil.rmtree(temp_results)

All solutions saved to ../outputs/alternative_weighting/test_solution.csv
Deleting temp directory _tmp/
